O notebook calcula a permissividade elétrica de misturas de etanol, levando em conta sua dependência da frequência. As estimativas são então exportadas em um arquivo .tab para serem importadas no HFSS.



In [ ]:
# Importa as bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.polynomial.polynomial import Polynomial
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
import os
import zipfile
import shutil

In [ ]:
# Create input fields for frequency range
text_freq_start = widgets.FloatText(value=6.0, description='Start Frequency (GHz):')
text_freq_end = widgets.FloatText(value=8.0, description='End Frequency (GHz):')
text_freq_step = widgets.FloatText(value=1.0, description='Step Frequency (MHz):')

# Function to update freq and omega based on input
def update_frequency_range(b):
    try:
        start_freq_ghz = text_freq_start.value
        end_freq_ghz = text_freq_end.value
        step_freq_mhz = text_freq_step.value

        # Convert to Hz
        start_freq_hz = start_freq_ghz * 1e9
        end_freq_hz = end_freq_ghz * 1e9
        step_freq_hz = step_freq_mhz * 1e6

        # Define frequency range
        global freq, omega
        freq = np.arange(start_freq_hz, end_freq_hz + step_freq_hz/2, step_freq_hz)
        omega = 2 * np.pi * freq

        print(f"Frequency range updated from {start_freq_ghz} GHz to {end_freq_ghz} GHz with step {step_freq_mhz} MHz.")

    except ValueError:
        print("Invalid input. Please enter numerical values for frequencies.")

# Add a button to update the frequency range
button_update = widgets.Button(description="Update Frequency Range")
button_update.on_click(update_frequency_range)

# Display the input fields and button
display(text_freq_start, text_freq_end, text_freq_step, button_update)

In [ ]:
# Create input fields for ethanol fraction range
text_fraction_start = widgets.FloatText(value=0.0, description='Start Fraction:')
text_fraction_end = widgets.FloatText(value=1.0, description='End Fraction:')
text_fraction_step = widgets.FloatText(value=0.02, description='Step Fraction:')

# Function to update fractions based on input
def update_fractions(b):
    try:
        start_fraction = text_fraction_start.value
        end_fraction = text_fraction_end.value
        step_fraction = text_fraction_step.value

        # Define fractions range
        global fractions
        fractions = np.round(np.arange(start_fraction, end_fraction + step_fraction/2, step_fraction), 2)

        print(f"Ethanol fractions updated from {start_fraction} to {end_fraction} with step {step_fraction}.")
        # Display the updated fractions (optional)
        # print(fractions)

    except ValueError:
        print("Invalid input. Please enter numerical values for fractions.")

# Add a button to update the fractions
button_update_fractions = widgets.Button(description="Update Fractions")
button_update_fractions.on_click(update_fractions)

# Display the input fields and button
display(text_fraction_start, text_fraction_end, text_fraction_step, button_update_fractions)

In [ ]:
# Modelo de Debye para gasolina e etanol
materiais = {
    "Gasolina": [2.12, 1.94, 3.18e-12],
    "Etanol":   [24.35, 2.12, 161e-12],
}

eps_s_gas, eps_inf_gas, tau_gas = materiais["Gasolina"]
eps_s_eta, eps_inf_eta, tau_eta = materiais["Etanol"]

eps_gas = eps_inf_gas + (eps_s_gas - eps_inf_gas) / (1 + 1j * omega * tau_gas)
eps_eta = eps_inf_eta + (eps_s_eta - eps_inf_eta) / (1 + 1j * omega * tau_eta)

# Dicionários para armazenar resultados
mistura_real = {}
mistura_tand = {}

for v_eta in fractions:
    v_gas = 1 - v_eta

    # Modelo cúbico de mistura
    eps_mix = (v_eta * np.power(eps_eta, 1/3) + v_gas * np.power(eps_gas, 1/3)) ** 3

    # Separação das partes real e imaginária
    eps_real = np.real(eps_mix)
    eps_imag = np.imag(eps_mix)
    tan_delta = np.abs(eps_imag) / eps_real

    key = f"{int(v_eta * 100):03d}"  # Ex: '000', '010', ..., '100'
    mistura_real[key] = eps_real
    mistura_tand[key] = tan_delta

In [ ]:
# Dicionários para armazenar resultados
mistura_real = {}     # Parte real da permissividade
mistura_tand = {}     # Tangente de perdas

for v_eta in fractions:
    v_gas = 1 - v_eta

    # Modelo cúbico de mistura
    eps_mix = (v_eta * np.power(eps_eta, 1/3) + v_gas * np.power(eps_gas, 1/3)) ** 3

    # Separação das partes real e imaginária
    eps_real = np.real(eps_mix)
    eps_imag = np.imag(eps_mix)
    tan_delta = np.abs(eps_imag) / eps_real

    # Armazenar nos dicionários
    key = f"{int(v_eta * 100):03d}"  # Ex: '000', '010', ..., '100'
    mistura_real[key] = eps_real
    mistura_tand[key] = tan_delta


In [ ]:
def exportar_tab(v_eta_str):
    if v_eta_str not in mistura_real:
        print(f"Erro: fração '{v_eta_str}' não encontrada. Use '000', '010', ..., '100'.")
        return

    valor_porcentagem = int(v_eta_str)
    inteiro = valor_porcentagem // 10
    decimal = valor_porcentagem % 10
    nome_porcentagem = f"E{inteiro*10}_{decimal}"

    eps_r = mistura_real[v_eta_str]
    tand = mistura_tand[v_eta_str]

    # Gerar nomes de arquivos
    arquivo_eps = f"eps_{nome_porcentagem}.tab"
    arquivo_tand = f"tan_delta_{nome_porcentagem}.tab"

    # Salvar ambos com notação científica e 6 casas decimais
    np.savetxt(arquivo_eps, np.column_stack((freq, eps_r)), delimiter="\t", fmt="%.6e")
    np.savetxt(arquivo_tand, np.column_stack((freq, tand)), delimiter="\t", fmt="%.6e")


    print(f"Arquivos exportados:\n- {arquivo_eps}\n- {arquivo_tand}")


In [ ]:
exportar_tab("010")

In [ ]:
# Parâmetros do modelo de Debye
materiais = {
    "Gasolina": [2.12, 1.94, 3.18e-12],
    "Etanol":   [24.35, 2.12, 161e-12],
}

eps_s_gas, eps_inf_gas, tau_gas = materiais["Gasolina"]
eps_s_eta, eps_inf_eta, tau_eta = materiais["Etanol"]

# Cálculo da permissividade complexa
eps_gas = eps_inf_gas + (eps_s_gas - eps_inf_gas) / (1 + 1j * omega * tau_gas)
eps_eta = eps_inf_eta + (eps_s_eta - eps_inf_eta) / (1 + 1j * omega * tau_eta)

# Dicionários para armazenar os resultados
mistura_real = {}
mistura_tand = {}

# Cálculo da mistura e extração de parte real e tangente de perdas
for v_eta in fractions:
    v_gas = 1 - v_eta
    eps_mix = (v_eta * np.power(eps_eta, 1/3) + v_gas * np.power(eps_gas, 1/3)) ** 3

    eps_real = np.real(eps_mix)
    eps_imag = np.imag(eps_mix)
    tan_delta = np.abs(eps_imag) / eps_real

    key = f"{round(v_eta * 100):03d}"
    mistura_real[key] = eps_real
    mistura_tand[key] = tan_delta

# Exportação de arquivos individuais para uma dada mistura
def exportar_tab(v_eta_str):
    if v_eta_str not in mistura_real:
        print(f"Erro: fração '{v_eta_str}' não encontrada. Use '000', '010', ..., '100'.")
        return

    valor_porcentagem = int(v_eta_str)
    inteiro = valor_porcentagem // 10
    decimal = valor_porcentagem % 10
    nome_porcentagem = f"E{inteiro*10}_{decimal}"

    eps_r = mistura_real[v_eta_str]
    tand = mistura_tand[v_eta_str]

    arquivo_eps = f"eps_{nome_porcentagem}.tab"
    arquivo_tand = f"tan_delta_{nome_porcentagem}.tab"

    np.savetxt(arquivo_eps, np.column_stack((freq, eps_r)), delimiter="\t", fmt="%.6e")
    np.savetxt(arquivo_tand, np.column_stack((freq, tand)), delimiter="\t", fmt="%.6e")

    print(f"Arquivos exportados:\n- {arquivo_eps}\n- {arquivo_tand}")

# Formata nomes como "E0", "E50", "E100"
def formatar_nome_dir(v_eta_str):
    return f"E{int(v_eta_str)}"

# Exportação total em pastas e compactação em ZIP
def exportar_pastas_zip():
    pasta_base = "exportados"

    if os.path.exists(pasta_base):
        shutil.rmtree(pasta_base)
    os.makedirs(pasta_base)

    for v_eta_str in mistura_real.keys():
        nome_dir = formatar_nome_dir(v_eta_str)
        pasta_mistura = os.path.join(pasta_base, nome_dir)
        os.makedirs(pasta_mistura)

        eps_r = mistura_real[v_eta_str]
        tand = mistura_tand[v_eta_str]

        arq_eps = os.path.join(pasta_mistura, f"{nome_dir}_eps.tab")
        arq_tand = os.path.join(pasta_mistura, f"{nome_dir}_tan_delta.tab")

        np.savetxt(arq_eps, np.column_stack((freq, eps_r)), delimiter="\t", fmt="%.6e")
        np.savetxt(arq_tand, np.column_stack((freq, tand)), delimiter="\t", fmt="%.6e")

        print(f"✓ Arquivos salvos em: {pasta_mistura}")

    zip_path = f"{pasta_base}.zip"
    shutil.make_archive(pasta_base, 'zip', pasta_base)
    print(f"\n📦 ZIP gerado: {zip_path}")


In [ ]:
    # Executar
    exportar_pastas_zip()

In [ ]:
import matplotlib.pyplot as plt

# Lista ordenada de chaves (ex: ['000', '010', ..., '100'])
chaves = sorted(mistura_real.keys())

# Calculate the number of rows needed for 2 columns
num_fractions = len(chaves)
nrows = (num_fractions * 2 + 1) // 2 # Two plots per fraction, 2 columns

fig, axes = plt.subplots(nrows=nrows, ncols=2, figsize=(12, nrows * 2.5), sharex=True)
axes = axes.flatten()

for i, key in enumerate(chaves):
    nome = f"E{int(key)}"  # Ex: 'E0', 'E10', etc.

    # Plot ε′(f)
    ax_eps = axes[i * 2]
    ax_eps.plot(freq / 1e9, mistura_real[key])
    ax_eps.set_title(f"{nome} — ε′")
    ax_eps.set_ylabel("ε′")
    ax_eps.grid(True)

    # Plot tan δ(f)
    ax_tand = axes[i * 2 + 1]
    ax_tand.plot(freq / 1e9, mistura_tand[key])
    ax_tand.set_title(f"{nome} — tan δ")
    ax_tand.set_ylabel("tan δ")
    ax_tand.grid(True)

# Eixo x comum só no final
for ax in axes[-2:]:
    ax.set_xlabel("Frequência (GHz)")

plt.tight_layout()
plt.show()